In [1]:
%load_ext autoreload
%autoreload 2


## Libraries

In [ ]:
# Imports
import os
import multiprocessing as mp
import pkg_resources
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import riskmapjnr as rmj


In [3]:
# GDAL
os.environ["GDAL_CACHEMAX"] = "1024"


## Set user parameters

In [ ]:
project_name = "test"


In [ ]:
# Datos del area total
jurisdiction_expected_total_deforestation_ha = 20000
years_to_forecast = 4


In [ ]:
# Proyecto dentro de la jurisdiccion
project_aoi_path = ""


In [ ]:
##Mapa de riesgo
risk_map_path = ""
risk_map_defrate_table = ""


## Connect folders

In [10]:
root_folder: Path = Path.cwd().parent
downloads_folder: Path = root_folder / "data"
downloads_folder.mkdir(parents=True, exist_ok=True)


In [11]:
project_folder = downloads_folder / project_name
project_folder.mkdir(parents=True, exist_ok=True)


In [12]:
allocation_folder = project_folder / "allocation"
allocation_folder.mkdir(parents=True, exist_ok=True)


## Reproject project aoi

In [15]:
sys.path.append("..")
from component.script.geo_utils import reproject_shapefile
from component.script.geo_utils import calculate_utm_rioxarray


In [18]:
calculated_epsg = calculate_utm_rioxarray(risk_map_path)


In [19]:
project_borders_repro = allocation_folder / f"project_{project_name}_repro.shp"


In [20]:
reproject_shapefile(project_aoi_path, project_borders_repro, calculated_epsg)


In [21]:
import os

import numpy as np
from osgeo import gdal
import pandas as pd
import forestatrisk

# Local application imports
from forestatrisk.misc import progress_bar, makeblock

opj = os.path.join
opd = os.path.dirname


def allocate_deforestation(
    riskmap_juris_file,
    defor_rate_tab,
    defor_juris_ha,
    years_forecast,
    project_borders,
    output_file="defor_project.csv",
    defor_density_map=False,
    blk_rows=128,
    verbose=False,
):
    """Allocating deforestation.

    :param riskmap_juris_file: Raster file with classes of deforestation
      risk at the jurisdictional level.

    :param defor_rate_tab: CSV file including the table with
      deforestation rates for each deforestation class.

    :param defor_juris_ha: Expected deforestation at the
      jurisdictional level (in hectares).

    :param years_forecast: Length of the forecasting period (in years).

    :param project_borders: Vector file for project borders.

    :param output_file: Output file with deforestation
      allocated to the project.

    :param defor_density_map: Compute the deforestation density map
      for the jurisdiction. Deforestation density is provided in
      ha/pixel/year (hectares of deforestation per pixel per year).
      Deforestation densities are floating-point numbers. For large
      jurisdictions (e.g. country scale) and high resolutions (e.g. 30
      m), this will produce a large raster file which will occupy
      a large amount of space on disk (e.g. several gigabytes).

    :param blk_rows: If > 0, number of rows for block (else 256x256).

    :param verbose: If True, print messages.

    """

    # Callback
    cback = gdal.TermProgress_nocb if verbose else 0

    # Creation options
    copts = ["COMPRESS=DEFLATE", "BIGTIFF=YES"]

    # ---------------------------------------
    # Crop riskmap to project boundaries
    # ---------------------------------------

    out_dir = opd(output_file)
    ofile = opj(out_dir, "project_riskmap.tif")
    gdal.Warp(
        ofile,
        riskmap_juris_file,
        cropToCutline=True,
        warpOptions=["CUTLINE_ALL_TOUCHED=TRUE"],
        cutlineDSName=project_borders,
        creationOptions=copts,
        callback=cback,
    )

    # ---------------------------------------
    # Compute number of pixels for each class
    # ---------------------------------------

    nvalues = 65535
    with gdal.Open(ofile) as ds:
        band = ds.GetRasterBand(1)
        counts = band.GetHistogram(0.5, 65535.5, nvalues, 0, 0)
    data = {"cat": [i + 1 for i in range(65535)], "counts": counts}
    df_count = pd.DataFrame(data)

    # Upload deforestation rates
    df_rate = pd.read_csv(defor_rate_tab)

    # -----------------------------
    # Compute deforestation density
    # -----------------------------

    # Pixel area
    pixel_area = df_rate.loc[0, "pixel_area"]

    # Correction factor, either ndefor / sum_i p_i
    # or theta * nfor / sum_i p_i
    sum_pi = (df_rate["nfor"] * df_rate["rate_mod"]).sum()
    correction_factor = defor_juris_ha / (pixel_area * sum_pi)

    # Absolute deforestation rate
    df_rate["rate_abs"] = df_rate["rate_mod"] * correction_factor

    # Deforestation density (ha/pixel/yr)
    df_rate["defor_dens"] = df_rate["rate_abs"] * pixel_area / years_forecast

    # Save the df_rate table
    ofile = opj(out_dir, "defrate_cat_forecast.csv")
    df_rate.to_csv(ofile)

    # -----------------------------
    # Join tables
    # -----------------------------

    df_project = df_count.merge(right=df_rate, on="cat", how="left")

    # Annual deforestation (ha) for project
    defor_project = (df_project["counts"] * df_project["defor_dens"]).sum()

    # Save results
    data = {
        "period": ["annual", "entire"],
        "length (yr)": [1, years_forecast],
        "deforestation (ha)": [
            round(defor_project, 1),
            round(defor_project * years_forecast, 1),
        ],
    }
    res = pd.DataFrame(data)
    res.to_csv(output_file, header=True, index=False)

    # -----------------------------
    # Get deforestation density map
    # -----------------------------

    if defor_density_map:
        riskmap_r = gdal.Open(riskmap_juris_file)
        riskmap_b = riskmap_r.GetRasterBand(1)
        gt = riskmap_r.GetGeoTransform()
        proj = riskmap_r.GetProjection()
        ncol = riskmap_r.RasterXSize
        nrow = riskmap_r.RasterYSize

        output_file = opj(out_dir, "deforestation_density_map.tif")
        driver = gdal.GetDriverByName("GTiff")
        if os.path.isfile(output_file):
            os.remove(output_file)
        ddm_r = driver.Create(
            output_file,
            ncol,
            nrow,
            1,
            gdal.GDT_Float64,
            ["COMPRESS=DEFLATE", "BIGTIFF=YES"],
        )
        ddm_r.SetGeoTransform(gt)
        ddm_r.SetProjection(proj)
        ddm_b = ddm_r.GetRasterBand(1)
        ddm_b.SetNoDataValue(-9999.0)

        # Make blocks
        blockinfo = makeblock(riskmap_juris_file, blk_rows=blk_rows)
        nblock = blockinfo[0]
        nblock_x = blockinfo[1]
        x = blockinfo[3]
        y = blockinfo[4]
        nx = blockinfo[5]
        ny = blockinfo[6]
        if verbose:
            print(f"Divide region in {nblock} blocks")

        # Write raster of dd
        if verbose:
            print("Write deforestation density raster")
        # Loop on blocks of data
        for b in range(nblock):
            # Progress bar
            progress_bar(nblock, b + 1)
            # Position in 1D-arrays
            px = b % nblock_x
            py = b // nblock_x
            # Data for one block
            risk_data = riskmap_b.ReadAsArray(x[px], y[py], nx[px], ny[py])
            risk_data = risk_data.flatten(order="C")
            # Get defor density from risk class
            defor_dens = np.zeros(len(risk_data))
            defor_dens[risk_data == 0] = -9999.0

            non_zero_risk_classes = risk_data[risk_data != 0]

            # Make sure df_rate has 'cat' as index (do this once after reading)
            df_rate_indexed = df_rate.set_index("cat")

            defor_dens_values = pd.Series(non_zero_risk_classes).map(
                df_rate_indexed["defor_dens"]
            )

            if defor_dens_values.isna().any():
                missing = non_zero_risk_classes[defor_dens_values.isna()]
                print(
                    f"Warning: Missing deforestation density for risk classes: {missing.unique()}"
                )

            # Fill missing with 0 (or -9999, or raise an error)
            defor_dens_values = defor_dens_values.fillna(0.0)

            defor_dens[risk_data != 0] = defor_dens_values.values
            defor_dens = defor_dens.reshape((ny[py], nx[px]), order="C")
            # Write deforestation densities
            ddm_b.WriteArray(defor_dens, x[px], y[py])

        # Compute statistics
        if verbose:
            print("Compute statistics")
        ddm_b.FlushCache()  # Write cache data to disk
        ddm_b.ComputeStatistics(False)

        # Dereference gdal datasets
        riskmap_b = None
        ddm_b = None
        del riskmap_r, ddm_r


## Allocate deforestation to project

In [22]:
import forestatrisk as far

allocate_deforestation(
    riskmap_juris_file=risk_map_path,
    defor_rate_tab=risk_map_defrate_table,
    defor_juris_ha=jurisdiction_expected_total_deforestation_ha,
    years_forecast=years_to_forecast,
    project_borders=project_borders_repro,
    output_file=allocation_folder / "defor_project.csv",
    defor_density_map=True,
    blk_rows=256,
    verbose=False,
)


10...20...30...40...50...60...70...80...90...100 - done
